In [ ]:
import os
import random
import pandas as pd
import numpy as np
from scipy.signal import find_peaks
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupShuffleSplit, train_test_split
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import classification_report, f1_score
import math
from numpy.lib.stride_tricks import sliding_window_view

SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
# -------------------------

data_folder = "data"
target_length = 100
batch_size = 32
learning_rate = 0.0001
num_epochs = 10
patience = 10


def get_label_from_filename(filename):
    if "Co" in filename:
        return 0
    elif "Pt" in filename:
        return 1
    else:
        return -1

def get_subject_id(filename):
    return filename.split("_")[0]

def sample_entropy(signal, m=2, r=0.2):
    N = len(signal)
    if N <= m + 1:
        return 0
    def _phi(m):
        x = sliding_window_view(signal, m)
        C = np.sum(np.max(np.abs(x[:, None] - x[None, :]), axis=2) < r * np.std(signal), axis=0) - 1
        return np.sum(C) / (N - m)
    return -np.log(_phi(m + 1) / (_phi(m) + 1e-10))

def petrosian_fd(signal):
    diff = np.diff(signal)
    N_delta = np.sum(diff[1:] * diff[:-1] < 0)
    n = len(signal)
    return math.log10(n) / (math.log10(n) + math.log10(n / (n + 0.4 * N_delta)))

df_list, labels, subject_ids = [], [], []
for root, _, files in os.walk(data_folder):
    for file in sorted(files):
        if file.endswith(".txt"):
            path = os.path.join(root, file)
            try:
                df_temp = pd.read_csv(path, delim_whitespace=True, header=None)
                if df_temp.shape[1] == 19:
                    df_temp.columns = [
                        "Time", "Fx_L", "Fy_L", "Fz_L", "COPx_L", "COPy_L", "Mx_L", "My_L", "Mz_L",
                        "Fx_R", "Fy_R", "Fz_R", "COPx_R", "COPy_R", "Mx_R", "My_R", "Mz_R",
                        "Other1", "Other2"]
                    label = get_label_from_filename(file)
                    if label == -1:
                        continue
                    df_list.append((df_temp, file))
                    labels.append(label)
                    subject_ids.append(get_subject_id(file))
            except Exception:
                continue

def segment_signal(signal, peaks, length=target_length):
    return np.array([
        np.interp(np.linspace(0, len(signal[peaks[i]:peaks[i+1]])-1, length),
                  np.arange(len(signal[peaks[i]:peaks[i+1]])),
                  signal[peaks[i]:peaks[i+1]])
        for i in range(len(peaks)-1) if len(signal[peaks[i]:peaks[i+1]]) >= 10
    ])

all_grf, all_cop, all_handcrafted, stride_labels, stride_subjects = [], [], [], [], []











for i, (df, fname) in enumerate(df_list):
    fz_l, fz_r = df["Fz_L"].values, df["Fz_R"].values
    copx_l, copy_l = df["COPx_L"].values, df["COPy_L"].values
    copx_r, copy_r = df["COPx_R"].values, df["COPy_R"].values
    
    peaks_l = find_peaks(fz_l, height=300, distance=80)[0]
    peaks_r = find_peaks(fz_r, height=300, distance=80)[0]
    
    grf_l = segment_signal(fz_l, peaks_l)
    grf_r = segment_signal(fz_r, peaks_r)
    cop_l_x = segment_signal(copx_l, peaks_l)
    cop_l_y = segment_signal(copy_l, peaks_l)
    cop_r_x = segment_signal(copx_r, peaks_r)
    cop_r_y = segment_signal(copy_r, peaks_r)
    
    if len(grf_l) == 0 or len(grf_r) == 0:
        continue
    
    # FIXED: Proper stacking for CNN input (channels, sequence_length)
    cop_l = np.stack([cop_l_x, cop_l_y], axis=2)  # Shape: (strides, 100, 2)
    cop_r = np.stack([cop_r_x, cop_r_y], axis=2)  # Shape: (strides, 100, 2)
    
    strides = min(len(grf_l), len(grf_r), len(cop_l), len(cop_r))
    if strides == 0:
        continue
        
    grf = (grf_l[:strides] + grf_r[:strides]) / 2
    cop = (cop_l[:strides] + cop_r[:strides]) / 2  # Shape: (strides, 100, 2)
    
    all_grf.append(grf)
    all_cop.append(cop)  # Will be (strides, 100, 2)
    stride_labels.extend([labels[i]] * strides)
    stride_subjects.extend([subject_ids[i]] * strides)










    def extract_feats(arr):
        return np.stack([np.mean(arr, axis=1), np.std(arr, axis=1),
                         skew(arr, axis=1), kurtosis(arr, axis=1)], axis=1) if arr.ndim == 2 else np.zeros((arr.shape[0], 4))
    handcrafted = np.hstack([
        extract_feats(grf_l[:strides]), extract_feats(grf_r[:strides]),
        extract_feats(cop_l_x[:strides]), extract_feats(cop_l_y[:strides]),
        extract_feats(cop_r_x[:strides]), extract_feats(cop_r_y[:strides])
    ])
    if len(peaks_l) > strides:
        stride_times = np.diff(peaks_l[:strides+1])
        mean_stride = np.mean(stride_times)
        std_stride = np.std(stride_times)
        cv_stride = std_stride / mean_stride if mean_stride != 0 else 0
    else:
        mean_stride = std_stride = cv_stride = 0
    irregular_feats = []
    for j in range(strides):
        segment = fz_l[peaks_l[j]:peaks_l[j+1]] if j+1 < len(peaks_l) else None
        if segment is not None and len(segment) > 10:
            sampen = sample_entropy(segment)
            fractal = petrosian_fd(segment)
        else:
            sampen = fractal = 0
        irregular_feats.append([mean_stride, std_stride, cv_stride, sampen, fractal])
    irregular_feats = np.array(irregular_feats)
    all_handcrafted.append(np.hstack([handcrafted, irregular_feats]))

X_grf_np = np.vstack(all_grf)
X_cop_np = np.vstack(all_cop)
X_hand_np = np.vstack(all_handcrafted)
y_np = np.array(stride_labels)
groups = np.array(stride_subjects)

def normalize_grf_all(train_array, val_array, test_array):
    mean = train_array.mean(axis=0, keepdims=True)
    std = train_array.std(axis=0, keepdims=True) + 1e-8
    train_norm = (train_array - mean) / std
    val_norm = (val_array - mean) / std
    test_norm = (test_array - mean) / std
    return train_norm, val_norm, test_norm

def normalize_cop_all(train_cop, val_cop, test_cop):
    """
    Normalize COP data properly for CNN input
    Input shape: (samples, sequence_length, channels) -> (samples, 100, 2)
    Output shape: (samples, sequence_length, channels) -> (samples, 100, 2)
    """
    # Reshape to (samples, features) for normalization
    train_reshaped = train_cop.reshape(train_cop.shape[0], -1)  # (samples, 200)
    val_reshaped = val_cop.reshape(val_cop.shape[0], -1)
    test_reshaped = test_cop.reshape(test_cop.shape[0], -1)
    
    # Calculate normalization parameters from training data only
    mean = train_reshaped.mean(axis=0, keepdims=True)
    std = train_reshaped.std(axis=0, keepdims=True) + 1e-8
    
    # Apply normalization
    train_norm = (train_reshaped - mean) / std
    val_norm = (val_reshaped - mean) / std
    test_norm = (test_reshaped - mean) / std
    
    # Reshape back to original shape
    return (train_norm.reshape(train_cop.shape), 
            val_norm.reshape(val_cop.shape), 
            test_norm.reshape(test_cop.shape))

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(X_hand_np, y_np, groups))
train_idx, val_idx = train_test_split(train_idx, test_size=0.1, random_state=42, stratify=y_np[train_idx])
X_grf_train_np, X_grf_val_np, X_grf_test_np = X_grf_np[train_idx], X_grf_np[val_idx], X_grf_np[test_idx]
X_cop_train_np, X_cop_val_np, X_cop_test_np = X_cop_np[train_idx], X_cop_np[val_idx], X_cop_np[test_idx]
X_hand_train_np, X_hand_val_np, X_hand_test_np = X_hand_np[train_idx], X_hand_np[val_idx], X_hand_np[test_idx]
y_train_np, y_val_np, y_test_np = y_np[train_idx], y_np[val_idx], y_np[test_idx]

scaler_hand = StandardScaler()
X_hand_train_np = scaler_hand.fit_transform(X_hand_train_np)
X_hand_val_np = scaler_hand.transform(X_hand_val_np)
X_hand_test_np = scaler_hand.transform(X_hand_test_np)

X_grf_train_np, X_grf_val_np, X_grf_test_np = normalize_grf_all(X_grf_train_np, X_grf_val_np, X_grf_test_np)

X_cop_train_np, X_cop_val_np, X_cop_test_np = normalize_cop_all(X_cop_train_np, X_cop_val_np, X_cop_test_np)

X_grf_train = torch.tensor(X_grf_train_np, dtype=torch.float32).unsqueeze(1)
X_grf_val = torch.tensor(X_grf_val_np, dtype=torch.float32).unsqueeze(1)
X_grf_test = torch.tensor(X_grf_test_np, dtype=torch.float32).unsqueeze(1)
X_cop_train = torch.tensor(X_cop_train_np.transpose(0, 2, 1), dtype=torch.float32)
X_cop_val = torch.tensor(X_cop_val_np.transpose(0, 2, 1), dtype=torch.float32)
X_cop_test = torch.tensor(X_cop_test_np.transpose(0, 2, 1), dtype=torch.float32)
print(f"COP tensor shapes after transpose:")
print(f"Train: {X_cop_train.shape}")  # Should be (n_samples, 2, 100)
print(f"Val: {X_cop_val.shape}")
print(f"Test: {X_cop_test.shape}")
X_hand_train = torch.tensor(X_hand_train_np, dtype=torch.float32)
X_hand_val = torch.tensor(X_hand_val_np, dtype=torch.float32)
X_hand_test = torch.tensor(X_hand_test_np, dtype=torch.float32)
y_train = torch.tensor(y_train_np, dtype=torch.long)
y_val = torch.tensor(y_val_np, dtype=torch.long)
y_test = torch.tensor(y_test_np, dtype=torch.long)

train_loader = DataLoader(TensorDataset(X_grf_train, X_cop_train, X_hand_train, y_train), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(X_grf_val, X_cop_val, X_hand_val, y_val), batch_size=batch_size)
test_loader = DataLoader(TensorDataset(X_grf_test, X_cop_test, X_hand_test, y_test), batch_size=batch_size)

class DualStreamModel(nn.Module):
    def __init__(self, hand_feat_dim):
        super().__init__()
        self.grf_cnn = nn.Sequential(
            nn.Conv1d(1, 16, 3, padding=1), nn.ReLU(), nn.MaxPool1d(2),
            nn.Conv1d(16, 32, 3, padding=1), nn.ReLU(), nn.MaxPool1d(2), nn.Flatten())
        self.cop_cnn = nn.Sequential(
            nn.Conv1d(2, 16, 3, padding=1), nn.ReLU(), nn.MaxPool1d(2),
            nn.Conv1d(16, 32, 3, padding=1), nn.ReLU(), nn.MaxPool1d(2), nn.Flatten())
        self.fc_hand = nn.Sequential(
            nn.Linear(hand_feat_dim, 64), nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 64), nn.ReLU(),
            nn.Dropout(0.3))
        with torch.no_grad():
            dummy_grf = torch.zeros(1, 1, target_length)
            dummy_cop = torch.zeros(1, 2, target_length)
            grf_feat_size = self.grf_cnn(dummy_grf).shape[1]
            cop_feat_size = self.cop_cnn(dummy_cop).shape[1]
        self.classifier = nn.Sequential(
            nn.Linear(grf_feat_size + cop_feat_size + 64, 128), nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 2))
    def forward(self, grf, cop, hand):
        grf_feat = self.grf_cnn(grf)
        cop_feat = self.cop_cnn(cop)
        hand_feat = self.fc_hand(hand)
        combined = torch.cat([grf_feat, cop_feat, hand_feat], dim=1)
        return self.classifier(combined)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DualStreamModel(X_hand_train.shape[1]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

def train_one_epoch():
    model.train()
    preds, trues = [], []
    for grf, cop, hand, labels in train_loader:
        grf, cop, hand, labels = grf.to(device), cop.to(device), hand.to(device), labels.to(device)
        optimizer.zero_grad()
        out = model(grf, cop, hand)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        preds += torch.argmax(out, 1).cpu().tolist()
        trues += labels.cpu().tolist()
    f1 = f1_score(trues, preds, average='macro')
    print(f"Train F1: {f1:.4f}")
    return f1

def evaluate(loader, split_name="Val"):
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for grf, cop, hand, labels in loader:
            grf, cop, hand = grf.to(device), cop.to(device), hand.to(device)
            out = model(grf, cop, hand)
            preds += torch.argmax(out, 1).cpu().tolist()
            trues += labels.tolist()
    print(f"\n{split_name} Classification Report:")
    print(classification_report(trues, preds, digits=4))
    f1 = f1_score(trues, preds, average='macro')
    print(f"{split_name} F1: {f1:.4f}\n")
    return f1

best_val_f1 = 0
patience_counter = 0
for epoch in range(10):
    print(f"Epoch {epoch+1}/{num_epochs}")
    train_f1 = train_one_epoch()
    val_f1 = evaluate(val_loader, split_name="Val")
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        patience_counter = 0
        torch.save(model.state_dict(), "best_model.pth")
        print("Saved Best Model!\n")
    else:  
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered.")
            break

C:\Users\arnav\AppData\Local\Temp\ipykernel_16136\110555103.py:68: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_temp = pd.read_csv(path, delim_whitespace=True, header=None)
C:\Users\arnav\AppData\Local\Temp\ipykernel_16136\110555103.py:68: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_temp = pd.read_csv(path, delim_whitespace=True, header=None)
C:\Users\arnav\AppData\Local\Temp\ipykernel_16136\110555103.py:68: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_temp = pd.read_csv(path, delim_whitespace=True, header=None)
C:\Users\arnav\AppData\Local\Temp\ipykernel_16136\110555103.py:68: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future v

COP tensor shapes after transpose:
Train: torch.Size([1377, 2, 100])
Val: torch.Size([154, 2, 100])
Test: torch.Size([740, 2, 100])
Epoch 1/10
Train F1: 0.7112

Val Classification Report:
              precision    recall  f1-score   support

           0     0.7917    0.7600    0.7755        75
           1     0.7805    0.8101    0.7950        79

    accuracy                         0.7857       154
   macro avg     0.7861    0.7851    0.7853       154
weighted avg     0.7859    0.7857    0.7855       154

Val F1: 0.7853

Saved Best Model!

Epoch 2/10
Train F1: 0.8079

Val Classification Report:
              precision    recall  f1-score   support

           0     0.8971    0.8133    0.8531        75
           1     0.8372    0.9114    0.8727        79

    accuracy                         0.8636       154
   macro avg     0.8671    0.8624    0.8629       154
weighted avg     0.8664    0.8636    0.8632       154

Val F1: 0.8629

Saved Best Model!

Epoch 3/10
Train F1: 0.8438

Val